In [1]:
Experimentos:

In [43]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
import seaborn as sns

A continuación leemos los datasets en dataframes de Pandas.

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.

    FB: Fuerza bruta
    BT: Backtracking con ambas podas.
    BT-F: Backtracking solamente con poda por factibilidad.
    BT-O: Backtracking solamente con poda por optimalidad.
    DP: Programación dinámica.



In [50]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

df_densidad_alta_factibilidad = pd.read_csv("instancias/densidad-alta-factibilidad/indice.csv");
#df_densidad_baja = pd.read_csv("instancias/densidad-baja/indice.csv");
df_mejor_caso_bt = pd.read_csv("instancias/mejor-caso-bt/indice.csv");
#df_peor_caso_bt = pd.read_csv("instancias/peor-caso-bt/indice.csv");
#df_dinamica = pd.read_csv("instancias/dinamica/indice.csv");

In [45]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["./a.out", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return tiempo_de_ejecucion;

In [51]:
experimentos = [];

Experimento Simple:

In [41]:
for n in range(0, 1):
    fila_n = df_mejor_caso_bt.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["n"], fila_n["W"], "FB", fila_n["archivo"]]);

Experimento Backtracking densidad alta factibilidad:

In [21]:
bt_resultados = pd.read_csv("instancias/densidad-alta-factibilidad/indice.csv");
bt_resultados.head()

,dataset,instancia,n,W,archivo
0,densidad-alta-factibilidad,BT-F-DA-1,1,3,instancias/densidad-alta-factibilidad/BT-F-DA-...
1,densidad-alta-factibilidad,BT-F-DA-2,2,6,instancias/densidad-alta-factibilidad/BT-F-DA-...
2,densidad-alta-factibilidad,BT-F-DA-3,3,9,instancias/densidad-alta-factibilidad/BT-F-DA-...
3,densidad-alta-factibilidad,BT-F-DA-4,4,12,instancias/densidad-alta-factibilidad/BT-F-DA-...
4,densidad-alta-factibilidad,BT-F-DA-5,5,15,instancias/densidad-alta-factibilidad/BT-F-DA-...


In [47]:
cantidad_locales_factibilidad = 45

In [52]:
for n in range(0, cantidad_locales_factibilidad - 1):
    fila_n = df_densidad_alta_factibilidad.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["n"], fila_n["W"], "BT", fila_n["archivo"]]);

Ejectuar los experimentos:

In [53]:
columnas = ["dataset", "n", "#locales","W", "metodo", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    for i in range(0, T):
        tiempos.append(correr_experimento(experimento[4], experimento[5]));
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], experimento[4],tiempo]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);

'Experimento: 44/44'